In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/IwM/blood-vessels-detection/
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/IwM/blood-vessels-detection
data  logs  net.ipynb  preprocessing.ipynb


In [ ]:
import os
import matplotlib.pyplot as plt 
import cv2
from skimage import filters, exposure, color, feature
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.patheffects as path_effects

In [ ]:
def show_images(name, titleimg,*images):
    fig, axes = plt.subplots(ncols=len(images))
    fig.set_size_inches(20,16)
    for i, img in enumerate(images):
      axes[i].set_title(titleimg[i],fontsize=20)
      axes[i].imshow(img, cmap='gray')
      axes[i].axis('off')
    plt.subplots_adjust(top=0.9, bottom=0.02, left=0.02, right=0.97)
    plt.savefig(f'data/image-processing-results/{name}')
    plt.show()

def detect_ridges(gray, sigma=1.0,order='xy'):
    H_elems = hessian_matrix(gray, sigma=sigma, order=order)
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges

def histo_equalized(gray):
    img_eq = exposure.equalize_hist(gray)
    img_equalized = exposure.equalize_adapthist(gray, clip_limit=0.02)
    return img_equalized

def clahe_equalized(gray):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_equalized = clahe.apply(gray)
    return img_equalized

def adjust_gamma(img, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    img_equalized = cv2.LUT(np.array(img, dtype = np.uint8), table)
    return img_equalized

def make_mask(img):
    return 255- cv2.threshold(img, 20, 255, cv2. THRESH_BINARY_INV)[1]

def process_image(img):
    green = img[:,:,1]
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

    img_clahe = clahe_equalized(green)

    img_gamma_corr = adjust_gamma(img_clahe, 1.2)

    ridges = detect_ridges(img_gamma_corr, sigma=3.0,order="rc")
    ridges = ridges + ridges.min() *(-1)
    ridges = ridges/ridges.max()
    ridges = ridges*255
    ridges = ridges.astype('uint8')

    mask = make_mask(gray)

    masked = np.zeros(ridges.shape)
    masked[mask > 0] = ridges[mask > 0]

    blur  = cv2.medianBlur(masked.astype('uint8'), 5)
    blur = 255-blur

    thresh = cv2.threshold(blur,140,255,cv2.THRESH_BINARY_INV)[1]

    return rgb, thresh

def cm_data(name, true, pred):
    cm = confusion_matrix(true.flatten(),pred.flatten())
    [tn, fp], [fn, tp] = cm

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    ax = plt.subplot()
    
    data = f"""
sensitivity:  {sensitivity:.4f}\n
specificity: {specificity:.4f}\n
accuracy: {accuracy:.4f}\n
geometric mean: {np.sqrt(specificity * sensitivity):.4f}
    """
    print(data)
    plt.figtext(1,0.5,data, size=12, va='center')
    sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['no vessel', 'vessel']); ax.yaxis.set_ticklabels(['no vessel', 'vessel']);
    plt.savefig(f'data/image-processing-results/{name}_cm.png')

In [ ]:
train_path_image = "data/test/images/"
train_path_label = "data/test/labels/"
try:
  os.makedirs(f'data/image-processing-results/')
except:
  pass

images = sorted( [f"{train_path_image}{file}" for file in os.listdir(train_path_image) ])
images = [cv2.imread(img) for img in images ]

labels = sorted([f"{train_path_label}{file}" for file in os.listdir(train_path_label) ])
labels = [cv2.imread(label) for label in labels]

true_labels = []
predicted_labels = []
title = ["Zdjęcie", "Moja etykieta", "Etykieta"] 

for i in range(len(images)):
    name = f"Image_1{i}L.jpg"
    print(name)
    rgb, img = process_image(images[i])
    predicted_labels.append(img)
    true_labels.append(labels[i])
    # print(labels[i],img.shape)
    cm_data(name, labels[i][:,:,0],img)
    show_images(name,title,rgb, img, labels[i])
  

Output hidden; open in https://colab.research.google.com to view.